# EmojiGen

#### Data Ingestion

In [1]:
# Importing necessary libraries

import re
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
openmoji_df = pd.read_csv('../data/openmoji.csv')
emojipedia_df = pd.read_csv('../data/emojipedia.csv')
llm_df = pd.read_parquet('../data/llmemoji.parquet')

In [3]:
openmoji_df

,emoji,hexcode,group,subgroups,annotation,tags,openmoji_tags,openmoji_author,openmoji_date,skintone,skintone_combination,skintone_base_emoji,skintone_base_hexcode,unicode,order
0,😀,1F600,smileys-emotion,face-smiling,grinning face,"face, grin",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,1,1.0
1,😃,1F603,smileys-emotion,face-smiling,grinning face with big eyes,"face, mouth, open, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,2.0
2,😄,1F604,smileys-emotion,face-smiling,grinning face with smiling eyes,"eye, face, mouth, open, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,3.0
3,😁,1F601,smileys-emotion,face-smiling,beaming face with smiling eyes,"eye, face, grin, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,4.0
4,😆,1F606,smileys-emotion,face-smiling,grinning squinting face,"face, laugh, mouth, satisfied, smile",NaN,Emily Jäger,2018-04-18,NaN,NaN,NaN,NaN,0.6,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,⮼,2BBC,extras-unicode,symbol-other,overlapping black squares,NaN,NaN,loominade,2020-04-22,NaN,NaN,NaN,NaN,1.1,NaN
4280,⯃,2BC3,extras-unicode,symbol-other,horizontal black octagon,NaN,equilateral polygon,loominade,2020-04-17,NaN,NaN,NaN,NaN,5,NaN
4281,⯄,2BC4,extras-unicode,symbol-other,black octagon,NaN,equilateral polygon,loominade,2020-04-17,NaN,NaN,NaN,NaN,5,NaN
4282,⯪,2BEA,extras-unicode,symbol-other,star with left half black,NaN,half star,Alexander Müller,2020-11-09,NaN,NaN,NaN,NaN,11,NaN


In [4]:
openmoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  4284 non-null   object 
 1   hexcode                4284 non-null   object 
 2   group                  4284 non-null   object 
 3   subgroups              4284 non-null   object 
 4   annotation             4284 non-null   object 
 5   tags                   1907 non-null   object 
 6   openmoji_tags          392 non-null    object 
 7   openmoji_author        4284 non-null   object 
 8   openmoji_date          4284 non-null   object 
 9   skintone               1875 non-null   object 
 10  skintone_combination   2198 non-null   object 
 11  skintone_base_emoji    2198 non-null   object 
 12  skintone_base_hexcode  2198 non-null   object 
 13  unicode                3903 non-null   object 
 14  order                  3782 non-null   float64
dtypes: f

In [5]:
openmoji_df.nunique()

emoji                    4283
hexcode                  4284
group                      12
subgroups                 119
annotation               4183
tags                     1742
openmoji_tags             314
openmoji_author           114
openmoji_date             120
skintone                   25
skintone_combination        2
skintone_base_emoji       323
skintone_base_hexcode     323
unicode                    20
order                    3782
dtype: int64

In [6]:
openmoji_df.isnull().sum()


emoji                       0
hexcode                     0
group                       0
subgroups                   0
annotation                  0
tags                     2377
openmoji_tags            3892
openmoji_author             0
openmoji_date               0
skintone                 2409
skintone_combination     2086
skintone_base_emoji      2086
skintone_base_hexcode    2086
unicode                   381
order                     502
dtype: int64

In [7]:
emojipedia_df

,Group,Subgroup,Emoji,Title,DescribedBy,URL,Description,Codepoints Hex
0,smiley,Smiling & Affectionate,😀,Grinning Face,grinning-face,/grinning-face,"A yellow face with simple, open eyes and a bro...",U+1F600
1,smiley,Smiling & Affectionate,😃,Grinning Face with Big Eyes,grinning-face-with-big-eyes,/grinning-face-with-big-eyes,"A yellow face with smiling eyes and a broad, o...",U+1F603
2,smiley,Smiling & Affectionate,😄,Grinning Face with Smiling Eyes,grinning-face-with-smiling-eyes,/grinning-face-with-smiling-eyes,"A yellow face with smiling eyes and a broad, o...",U+1F604
3,smiley,Smiling & Affectionate,😁,Beaming Face with Smiling Eyes,beaming-face-with-smiling-eyes,/beaming-face-with-smiling-eyes,A yellow face with smiling eyes and full-tooth...,U+1F601
4,smiley,Smiling & Affectionate,😆,Grinning Squinting Face,grinning-squinting-face,/grinning-squinting-face,"A yellow face with a broad, open smile and scr...",U+1F606
...,...,...,...,...,...,...,...,...
1880,flags,"Oceania, Island Nations & Territories",🇫🇲,Flag: Micronesia,flag-micronesia,/flag-micronesia,"The flag for Micronesia , which may show as th...","U+1F1EB,U+1F1F2"
1881,objects,Clothing & Appearance,👒,Woman’s Hat,womans-hat,/womans-hat,"A hat worn by women for sun protection, or for...",U+1F452
1882,objects,Office & Stationery,📕,Closed Book,closed-book,/closed-book,"A closed, hardcover book depicted with a red c...",U+1F4D5
1883,objects,Tools & Household Items,📡,Satellite Antenna,satellite-antenna,/satellite-antenna,"A dish antenna, as used to send or receive inf...",U+1F4E1


In [8]:
emojipedia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Group           1885 non-null   object
 1   Subgroup        1885 non-null   object
 2   Emoji           1885 non-null   object
 3   Title           1885 non-null   object
 4   DescribedBy     1885 non-null   object
 5   URL             1885 non-null   object
 6   Description     1885 non-null   object
 7   Codepoints Hex  1885 non-null   object
dtypes: object(8)
memory usage: 117.9+ KB


In [9]:
emojipedia_df.nunique()

Group                9
Subgroup            49
Emoji             1885
Title             1885
DescribedBy       1885
URL               1885
Description       1880
Codepoints Hex    1885
dtype: int64

In [10]:
llm_df

,character,unicode,short description,tags,LLM description
0,🥇,U+1F947,1ST PLACE MEDAL,"[first place, victory, achievement, success, c...","This emoji represents a first place medal, oft..."
1,🥈,U+1F948,2ND PLACE MEDAL,"[medal, silver, second place, achievement, suc...","This emoji represents a silver medal, often us..."
2,🥉,U+1F949,3RD PLACE MEDAL,"[medal, bronze, third place, achievement, spor...","This emoji represents a bronze medal, symboliz..."
3,🆎,U+1F18E,AB BUTTON (BLOOD TYPE),"[blood type, AB, medical, compatibility, trans...",This emoji represents the AB blood type symbol...
4,🏧,U+1F3E7,ATM SIGN,"[ATM, banking, finance, money, transaction, lo...","This emoji represents an ATM sign, often used ..."
...,...,...,...,...,...
5029,🤪,U+1F92A,ZANY FACE,"[zany, playful, silly, humor, lightheartedness...","This emoji represents a zany face, expressing ..."
5030,🦓,U+1F993,ZEBRA,"[zebra, animal, stripes, mammal, nature, fast,...","This emoji represents a zebra, a fast and agil..."
5031,🤐,U+1F910,ZIPPER-MOUTH FACE,"[zipper-mouth, silence, secrecy, mute, speechl...",This emoji represents a face with a zipper ove...
5032,🧟,U+1F9DF,ZOMBIE,"[zombie, undead, horror, fiction, scary, reani...","This emoji represents a zombie, a fictional un..."


In [11]:
llm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5034 entries, 0 to 5033
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   character          5034 non-null   object
 1   unicode            5034 non-null   object
 2   short description  5034 non-null   object
 3   tags               5034 non-null   object
 4   LLM description    5034 non-null   object
dtypes: object(5)
memory usage: 196.8+ KB


In [12]:
llm_df[llm_df['unicode'] == 'U+1F947']

,character,unicode,short description,tags,LLM description
0,🥇,U+1F947,1ST PLACE MEDAL,"[first place, victory, achievement, success, c...","This emoji represents a first place medal, oft..."


In [13]:
emojipedia_df[emojipedia_df['Codepoints Hex'] == 'U+0023,U+FE0F,U+20E3']

,Group,Subgroup,Emoji,Title,DescribedBy,URL,Description,Codepoints Hex


In [14]:
openmoji_df[openmoji_df['openmoji_tags'] == 'logo']

,emoji,hexcode,group,subgroups,annotation,tags,openmoji_tags,openmoji_author,openmoji_date,skintone,skintone_combination,skintone_base_emoji,skintone_base_hexcode,unicode,order


In [15]:
openmoji_df['openmoji_tags'].unique()

array([nan, 'goldfish, fish, water', 'poppy, remembrance, veterans',
       'forest, hunt, antler, deer',
       'cetacean, whale, spouting, orca, killer whale, blackfish, swim, animal, water',
       'cetacean, whale, orca, killer whale, blackfish, swim, animal, water',
       'cetacean, whale, swim, animal, water',
       'cetacean, swim, animal, water', 'vermin, rock dove, bird, city',
       'tropical, jungle, exotic, parrots', 'amoeba, bacteria, virus',
       'social, platform, pins', 'facebook, company, social-media',
       'instagram, facebook, company, social-media',
       'social, platform, video, brand',
       'social, platform, code, git, brand', 'business', 'google play',
       'music metadata', 'nutrition data', 'map, osm, geodata',
       'open knowledge graph',
       'logo, net, dotnet, programming, framework',
       'logo, go, golang, google, programming, language',
       'logo, kotlin, jvm, programming, language',
       'logo, ruby, programming, language', 'br

In [16]:
def normalize_unicode(unicode_str):
    # Remove existing "U+" prefixes
    cleaned = re.sub(r"U\+", "", unicode_str)

    # Replace separators (spaces, commas, hyphens) with a single space
    cleaned = re.sub(r"[\s,.-]+", " ", cleaned)

    # Add "U+" prefix to each code point
    normalized = " ".join(f"U+{code}" for code in cleaned.split())

    return normalized

In [17]:
# Merge the llm_emoji_df, openmoji_df, emojipedia_df based on unicode/hexcode
# The final df should have unicode, title (short descriotion, annotation), tags, group, subgroup, description)
llm_df.info()
llm_sm_df = llm_df[['character','unicode','tags','LLM description']]
llm_sm_df.rename(columns={'character': 'llm_emoji','tags': 'llm_emoji_tags','LLM description': 'llm_description'}, inplace=True)
llm_sm_df['unicode'] = llm_sm_df['unicode'].apply(normalize_unicode)

llm_sm_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5034 entries, 0 to 5033
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   character          5034 non-null   object
 1   unicode            5034 non-null   object
 2   short description  5034 non-null   object
 3   tags               5034 non-null   object
 4   LLM description    5034 non-null   object
dtypes: object(5)
memory usage: 196.8+ KB


,llm_emoji,unicode,llm_emoji_tags,llm_description
0,🥇,U+1F947,"[first place, victory, achievement, success, c...","This emoji represents a first place medal, oft..."
1,🥈,U+1F948,"[medal, silver, second place, achievement, suc...","This emoji represents a silver medal, often us..."
2,🥉,U+1F949,"[medal, bronze, third place, achievement, spor...","This emoji represents a bronze medal, symboliz..."
3,🆎,U+1F18E,"[blood type, AB, medical, compatibility, trans...",This emoji represents the AB blood type symbol...
4,🏧,U+1F3E7,"[ATM, banking, finance, money, transaction, lo...","This emoji represents an ATM sign, often used ..."
...,...,...,...,...
5029,🤪,U+1F92A,"[zany, playful, silly, humor, lightheartedness...","This emoji represents a zany face, expressing ..."
5030,🦓,U+1F993,"[zebra, animal, stripes, mammal, nature, fast,...","This emoji represents a zebra, a fast and agil..."
5031,🤐,U+1F910,"[zipper-mouth, silence, secrecy, mute, speechl...",This emoji represents a face with a zipper ove...
5032,🧟,U+1F9DF,"[zombie, undead, horror, fiction, scary, reani...","This emoji represents a zombie, a fictional un..."


In [18]:
openmoji_df.info()
openmoji_sm_df = openmoji_df[['emoji','hexcode','group','subgroups','annotation','tags','openmoji_tags']]
openmoji_sm_df.rename(columns={'emoji':'openmoji_emoji','hexcode':'unicode', 'annotation':'openmoji_annotations'}, inplace=True)
openmoji_sm_df['unicode'] = openmoji_sm_df['unicode'].apply(normalize_unicode)

def merge_columns(col1, col2):
    if pd.isna(col1) and pd.isna(col2):  
        return float("NaN")  # Keep NaN if both are missing
    elif pd.isna(col1):  
        return col2  # If col1 is NaN, take col2
    elif pd.isna(col2):  
        return col1  # If col2 is NaN, take col1
    else:  
        return f"{col1},{col2}"  # If both have values, merge with a comma

openmoji_sm_df['openmoji_tags'] = openmoji_sm_df.apply(lambda row: merge_columns(row['tags'], row["openmoji_tags"]), axis=1)
openmoji_sm_df.drop(columns=["tags"], inplace=True)
openmoji_sm_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   emoji                  4284 non-null   object 
 1   hexcode                4284 non-null   object 
 2   group                  4284 non-null   object 
 3   subgroups              4284 non-null   object 
 4   annotation             4284 non-null   object 
 5   tags                   1907 non-null   object 
 6   openmoji_tags          392 non-null    object 
 7   openmoji_author        4284 non-null   object 
 8   openmoji_date          4284 non-null   object 
 9   skintone               1875 non-null   object 
 10  skintone_combination   2198 non-null   object 
 11  skintone_base_emoji    2198 non-null   object 
 12  skintone_base_hexcode  2198 non-null   object 
 13  unicode                3903 non-null   object 
 14  order                  3782 non-null   float64
dtypes: f

,openmoji_emoji,unicode,group,subgroups,openmoji_annotations,openmoji_tags
0,😀,U+1F600,smileys-emotion,face-smiling,grinning face,"face, grin"
1,😃,U+1F603,smileys-emotion,face-smiling,grinning face with big eyes,"face, mouth, open, smile"
2,😄,U+1F604,smileys-emotion,face-smiling,grinning face with smiling eyes,"eye, face, mouth, open, smile"
3,😁,U+1F601,smileys-emotion,face-smiling,beaming face with smiling eyes,"eye, face, grin, smile"
4,😆,U+1F606,smileys-emotion,face-smiling,grinning squinting face,"face, laugh, mouth, satisfied, smile"
...,...,...,...,...,...,...
4279,⮼,U+2BBC,extras-unicode,symbol-other,overlapping black squares,NaN
4280,⯃,U+2BC3,extras-unicode,symbol-other,horizontal black octagon,equilateral polygon
4281,⯄,U+2BC4,extras-unicode,symbol-other,black octagon,equilateral polygon
4282,⯪,U+2BEA,extras-unicode,symbol-other,star with left half black,half star


In [19]:
emojipedia_df.info()
emojipedia_sm_df = emojipedia_df[['Emoji','Description','Codepoints Hex']]
emojipedia_sm_df.rename(columns={'Emoji':'emojipedia_emoji','Codepoints Hex':'unicode', 'Description':'emojipedia_description'}, inplace=True)
emojipedia_sm_df['unicode'] = emojipedia_sm_df['unicode'].apply(normalize_unicode)
emojipedia_sm_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Group           1885 non-null   object
 1   Subgroup        1885 non-null   object
 2   Emoji           1885 non-null   object
 3   Title           1885 non-null   object
 4   DescribedBy     1885 non-null   object
 5   URL             1885 non-null   object
 6   Description     1885 non-null   object
 7   Codepoints Hex  1885 non-null   object
dtypes: object(8)
memory usage: 117.9+ KB


,emojipedia_emoji,emojipedia_description,unicode
0,😀,"A yellow face with simple, open eyes and a bro...",U+1F600
1,😃,"A yellow face with smiling eyes and a broad, o...",U+1F603
2,😄,"A yellow face with smiling eyes and a broad, o...",U+1F604
3,😁,A yellow face with smiling eyes and full-tooth...,U+1F601
4,😆,"A yellow face with a broad, open smile and scr...",U+1F606
...,...,...,...
1880,🇫🇲,"The flag for Micronesia , which may show as th...",U+1F1EB U+1F1F2
1881,👒,"A hat worn by women for sun protection, or for...",U+1F452
1882,📕,"A closed, hardcover book depicted with a red c...",U+1F4D5
1883,📡,"A dish antenna, as used to send or receive inf...",U+1F4E1


In [20]:
#Merging the datasets
final_df = llm_sm_df.merge(openmoji_sm_df, on='unicode', how='outer').merge(emojipedia_sm_df, on='unicode', how='outer')
final_df

,llm_emoji,unicode,llm_emoji_tags,llm_description,openmoji_emoji,group,subgroups,openmoji_annotations,openmoji_tags,emojipedia_emoji,emojipedia_description
0,NaN,U+0023 U+FE0F U+20E3,NaN,NaN,#️⃣,symbols,keycap,keycap: #,keycap,NaN,NaN
1,NaN,U+002A U+FE0F U+20E3,NaN,NaN,*️⃣,symbols,keycap,keycap: *,keycap,NaN,NaN
2,NaN,U+002D,NaN,NaN,-,extras-unicode,symbol-other,hyphen-minus,"hyphen, minus, dash, line",NaN,NaN
3,NaN,U+0030 U+FE0F U+20E3,NaN,NaN,0️⃣,symbols,keycap,keycap: 0,keycap,NaN,NaN
4,NaN,U+0031 U+FE0F U+20E3,NaN,NaN,1️⃣,symbols,keycap,keycap: 1,keycap,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5545,NaN,U+E380,NaN,NaN,,extras-openmoji,symbols,no handshaking,"hygiene, agreement, virus, meeting, spread, germs",NaN,NaN
5546,NaN,U+E381,NaN,NaN,,extras-openmoji,symbols,web syndication,"feed, RSS, atom feed, podcast, subscribe, web ...",NaN,NaN
5547,NaN,U+F000,NaN,NaN,,extras-openmoji,brand,windows,Microsoft,NaN,NaN
5548,NaN,U+F77A,NaN,NaN,,extras-openmoji,brand,artstation,"art, brand",NaN,NaN


In [21]:
def merge_columns(col1, col2):
    if pd.isna(col1) and pd.isna(col2):  
        return float("NaN")  # Keep NaN if both are missing
    elif pd.isna(col1):  
        return col2  # If col1 is NaN, take col2
    elif pd.isna(col2):  
        return col1  # If col2 is NaN, take col1
    else:  
        return f"{col1}"  # If both have values, replace with first

In [22]:
final_df['emoji'] = final_df.apply(lambda row: merge_columns(row['llm_emoji'], row["openmoji_emoji"]), axis=1)
final_df.drop(columns=["llm_emoji","openmoji_emoji"], inplace=True)
final_df['emoji'] = final_df.apply(lambda row: merge_columns(row['emoji'], row["emojipedia_emoji"]), axis=1)
final_df.drop(columns=["emojipedia_emoji"], inplace=True)

final_df

,unicode,llm_emoji_tags,llm_description,group,subgroups,openmoji_annotations,openmoji_tags,emojipedia_description,emoji
0,U+0023 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: #,keycap,NaN,#️⃣
1,U+002A U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: *,keycap,NaN,*️⃣
2,U+002D,NaN,NaN,extras-unicode,symbol-other,hyphen-minus,"hyphen, minus, dash, line",NaN,-
3,U+0030 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 0,keycap,NaN,0️⃣
4,U+0031 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 1,keycap,NaN,1️⃣
...,...,...,...,...,...,...,...,...,...
5545,U+E380,NaN,NaN,extras-openmoji,symbols,no handshaking,"hygiene, agreement, virus, meeting, spread, germs",NaN,
5546,U+E381,NaN,NaN,extras-openmoji,symbols,web syndication,"feed, RSS, atom feed, podcast, subscribe, web ...",NaN,
5547,U+F000,NaN,NaN,extras-openmoji,brand,windows,Microsoft,NaN,
5548,U+F77A,NaN,NaN,extras-openmoji,brand,artstation,"art, brand",NaN,


In [23]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5550 entries, 0 to 5549
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   unicode                 5550 non-null   object
 1   llm_emoji_tags          5034 non-null   object
 2   llm_description         5034 non-null   object
 3   group                   4284 non-null   object
 4   subgroups               4284 non-null   object
 5   openmoji_annotations    4284 non-null   object
 6   openmoji_tags           2299 non-null   object
 7   emojipedia_description  1885 non-null   object
 8   emoji                   5550 non-null   object
dtypes: object(9)
memory usage: 390.4+ KB


In [24]:
final_df.isnull().sum()

unicode                      0
llm_emoji_tags             516
llm_description            516
group                     1266
subgroups                 1266
openmoji_annotations      1266
openmoji_tags             3251
emojipedia_description    3665
emoji                        0
dtype: int64

In [25]:
from IPython.display import display, HTML

# Display the dataset in a scrollable box
display(HTML(final_df.to_html()))

,unicode,llm_emoji_tags,llm_description,group,subgroups,openmoji_annotations,openmoji_tags,emojipedia_description,emoji
0,U+0023 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: #,keycap,NaN,#️⃣
1,U+002A U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: *,keycap,NaN,*️⃣
2,U+002D,NaN,NaN,extras-unicode,symbol-other,hyphen-minus,"hyphen, minus, dash, line",NaN,-
3,U+0030 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 0,keycap,NaN,0️⃣
4,U+0031 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 1,keycap,NaN,1️⃣
5,U+0032 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 2,keycap,NaN,2️⃣
6,U+0033 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 3,keycap,NaN,3️⃣
7,U+0034 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 4,keycap,NaN,4️⃣
8,U+0035 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 5,keycap,NaN,5️⃣
9,U+0036 U+FE0F U+20E3,NaN,NaN,symbols,keycap,keycap: 6,keycap,NaN,6️⃣
